# Homework Starter — Stage 15: Orchestration & System Design
Complete the sections below. Keep your answers concise and focused on orchestration readiness.

## 1) Project Task Decomposition
List 4–8 tasks. Add more rows as needed.

In [1]:
from pathlib import Path
import pandas as pd
tasks = pd.DataFrame({
    'task': ['ingest', 'clean', 'train_or_score', 'report'],
    'inputs': ['/data/raw.ext', 'prices_raw.json', 'prices_clean.json', 'model.json'],
    'outputs': ['prices_raw.json', 'prices_clean.json', 'model.json', 'report.txt'],
    'idempotent': [True, True, True, True]
})
tasks

,task,inputs,outputs,idempotent
0,ingest,/data/raw.ext,prices_raw.json,True
1,clean,prices_raw.json,prices_clean.json,True
2,train_or_score,prices_clean.json,model.json,True
3,report,model.json,report.txt,True


## 2) Dependencies (DAG)
Describe dependencies and paste a small diagram if you have one.

In [2]:
dag = {
    'ingest': [],
    'clean': ['ingest'],
    'train_or_score': ['clean'],
    'report': ['train_or_score']
}
dag

{'ingest': [],
 'clean': ['ingest'],
 'train_or_score': ['clean'],
 'report': ['train_or_score']}

## 3) Logging & Checkpoints Plan
Specify what you will log and where you will checkpoint for each task.

In [3]:
logging_plan = pd.DataFrame({
    'task': ['ingest', 'clean', 'train_or_score', 'report'],
    'log_messages': [
        'start/end, rows, source URI',
        'start/end, rows in/out',
        'params, metrics',
        'artifact path'
    ],
    'checkpoint_artifact': [
        'prices_raw.json',
        'prices_clean.json',
        'model.json',
        'report.txt'
    ]
})
logging_plan

,task,log_messages,checkpoint_artifact
0,ingest,"start/end, rows, source URI",prices_raw.json
1,clean,"start/end, rows in/out",prices_clean.json
2,train_or_score,"params, metrics",model.json
3,report,artifact path,report.txt


## 4) Right-Sizing Automation
Which parts will you automate now? Which stay manual? Why?

For this project, some tasks are stable and repeatable enough to automate immediately, while others are better left manual until the workflow matures.

### Automate Now
- **Ingest**: Pulling raw data into `prices_raw.json` is straightforward and idempotent. Automating ensures consistent data updates without manual intervention.  
- **Clean**: Converting raw prices into `prices_clean.json` should also be automated, since the steps (removing nulls, formatting) are deterministic and repeatable.

### Keep Manual (for now)
- **Train_or_score**: Although it could be automated later, keeping this step manual initially allows monitoring of model quality and validation of training results before promoting a new model.  
- **Report**: Generating `report.txt` should stay manual for now, since interpretation of results may change, and reports may require human review for clarity and correctness.

### Rationale
Automating ingest and clean reduces repetitive work and ensures reproducibility of input data. Leaving training and reporting as manual steps gives flexibility to debug, validate metrics, and adjust reporting before committing to full automation. Once the pipeline stabilizes and performance thresholds are well defined, these steps can also be automated.

## 5) (Stretch) Refactor One Task into a Function + CLI
Use the templates below.

In [9]:
import argparse, json, logging, sys
from datetime import datetime

def my_task(input_path: str, output_path: str) -> None:
    '''Example task template: read → transform → write JSON.'''
    logging.info('[my_task] start')
    # TODO: implement your logic
    result = {'run_at': datetime.utcnow().isoformat(), 'note': 'replace with real output'}
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    Path(output_path).write_text(json.dumps(result, indent=2))
    logging.info('[my_task] wrote %s', output_path)

def main(argv=None):
    parser = argparse.ArgumentParser(description='Homework task wrapper')
    parser.add_argument('--input', required=True)
    parser.add_argument('--output', required=True)
    args = parser.parse_args(argv)
    logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)])
    my_task(args.input, args.output)

if __name__ == "__main__":
    # Example simulated CLI in notebook:
    main(['--input', 'data/in.ext', '--output', 'data/out.json'])

INFO:root:[my_task] start
INFO:root:[my_task] wrote data/out.json


### Optional: Simple Retry Wrapper (fill in)
Add a small retry with linear backoff to harden a task.

In [10]:
import time
def retry(n_tries=3, delay=0.2):
    def wrapper(fn, *args, **kwargs):
        # TODO: implement try/except loop with sleep backoff
        def wrapper(*args, **kwargs):
            for attempt in range(1, n_tries + 1):
                try:
                    return fn(*args, **kwargs)
                except Exception as e:
                    if attempt == n_tries:
                        raise  # re-raise the last error
                    wait = delay * attempt
                    print(f"[retry] Attempt {attempt} failed with {e}, retrying in {wait:.1f}s...")
                    time.sleep(wait)
        return fn(*args, **kwargs)
    return wrapper